<a href="https://colab.research.google.com/github/parlakisik/packetinspector/blob/main/ai/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: show files in current directory

!ls -al train_data/


total 137396
drwxr-xr-x 2 root root     4096 Apr 11 07:27  .
drwxr-xr-x 1 root root     4096 Apr 11 07:27  ..
-rw-r--r-- 1 root root     4044 Apr 11 07:27  NUSW-NB15_features.csv
-rw-r--r-- 1 root root   189112 Apr 11 07:27 'The UNSW-NB15 description.pdf'
-rw-r--r-- 1 root root 42775611 Apr 11 07:29  UNSW-NB15_1.csv.gz
-rw-r--r-- 1 root root 43244175 Apr 11 07:29  UNSW-NB15_2.csv.gz
-rw-r--r-- 1 root root 33168775 Apr 11 07:29  UNSW-NB15_3.csv.gz
-rw-r--r-- 1 root root 21277181 Apr 11 07:28  UNSW-NB15_4.csv.gz


Unzipping files

In [2]:
!gzip -d train_data/*.gz

Importing libraries

In [3]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten , Activation, SimpleRNN, LSTM, GRU, Dropout, TimeDistributed, Reshape, Input, Lambda, Add
from tensorflow.keras import Sequential
from sklearn.preprocessing import LabelEncoder


Loading test data

In [4]:
dfs = []
for i in range(1,5):
    path = 'train_data/UNSW-NB15_{}.csv'
    dfs.append(pd.read_csv(path.format(i), header = None))
combined_data = pd.concat(dfs).reset_index(drop=True)

<ipython-input-4-4f7193ad3560>:4: DtypeWarning:

Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.

<ipython-input-4-4f7193ad3560>:4: DtypeWarning:

Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
combined_data.shape

(2540047, 49)

In [6]:
combined_data.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [7]:
dataset_columns = pd.read_csv('train_data/NUSW-NB15_features.csv',encoding='ISO-8859-1')
combined_data.columns = dataset_columns['Name']

In [8]:
combined_data.head()

Name,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


Data status

In [9]:
def print_data_status():
  print("Combined data labels")
  print(combined_data['Label'].value_counts())
  print("Combined data features")
  print(combined_data.columns)
  print("Number of null data")
  print(combined_data.isnull().sum())
  print("Number of attack_cat values")
  print(combined_data['attack_cat'].value_counts())
  for col_name in combined_data.columns:
    print("Col name values ",col_name)
    print(combined_data[col_name].value_counts())


In [10]:
print(combined_data.columns)

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object', name='Name')


In [11]:
print_data_status()

Combined data labels
Label
0    2218764
1     321283
Name: count, dtype: int64
Combined data features
Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object', name='Name')
Number of null data
Name
srcip                     0
sport                     0
dstip                     0
dsport                    0
proto                     0
state                     0
dur                       0
sbytes              

In [12]:
def fix_data():
  # Filling none attacks to no_attack
  combined_data['attack_cat'] = combined_data['attack_cat'].fillna(value='no_attack').apply(lambda x: x.strip().lower())
  # Fixing backdoors and back door
  combined_data['attack_cat'] = combined_data['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
  combined_data['attack_cat'] = combined_data['attack_cat'].replace(' Shellcode','Shellcode', regex=True).apply(lambda x: x.strip().lower())
  combined_data['attack_cat'] = combined_data['attack_cat'].replace(' Fuzzers','Fuzzers', regex=True).apply(lambda x: x.strip().lower())
  combined_data['attack_cat'] = combined_data['attack_cat'].replace(' Fuzzers ','Fuzzers', regex=True).apply(lambda x: x.strip().lower())
  combined_data['attack_cat'] = combined_data['attack_cat'].replace(' Reconnaissance','Reconnaissance', regex=True).apply(lambda x: x.strip().lower())
  # Fill null https methos to 0
  combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
  # Fix is ftp login to 0 and false
  combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
  combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])

  combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
  combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)


In [13]:
fix_data()

In [14]:
# Removing src and dst ip adresses
combined_data.drop(columns=['srcip','dstip','Label'],inplace=True)

In [15]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540047 entries, 0 to 2540046
Data columns (total 46 columns):
 #   Column            Dtype  
---  ------            -----  
 0   sport             object 
 1   dsport            object 
 2   proto             object 
 3   state             object 
 4   dur               float64
 5   sbytes            int64  
 6   dbytes            int64  
 7   sttl              int64  
 8   dttl              int64  
 9   sloss             int64  
 10  dloss             int64  
 11  service           object 
 12  Sload             float64
 13  Dload             float64
 14  Spkts             int64  
 15  Dpkts             int64  
 16  swin              int64  
 17  dwin              int64  
 18  stcpb             int64  
 19  dtcpb             int64  
 20  smeansz           int64  
 21  dmeansz           int64  
 22  trans_depth       int64  
 23  res_bdy_len       int64  
 24  Sjit              float64
 25  Djit              float64
 26  Stime         

In [16]:
for column in combined_data.select_dtypes(include='int64').columns:
    combined_data[column] = combined_data[column].astype(np.float64)


In [36]:
def convert_to_int(value):
    try:
        if type(value) == int:
            return value
        elif value.startswith('0x'):
            return int(value, 16)
        else:
            return int(value)
    except ValueError:
        return -1  # or return some default value if conversion fails

combined_data['sport'] = combined_data['sport'].apply(convert_to_int)
combined_data["dsport"] = combined_data["dsport"].apply(convert_to_int)
combined_data['sport'] = combined_data['sport'].astype(np.float64)
combined_data['dsport'] = combined_data['dsport'].astype(np.float64)

In [ ]:
for column in combined_data.select_dtypes(include='object').columns:
    print(column)
    protocol_lookup = tf.keras.layers.StringLookup(output_mode='int')
    protocol_lookup.adapt(combined_data[column])
    combined_data[column] = protocol_lookup(combined_data[column])
    combined_data[column] = combined_data[column].astype(np.float64)



proto


In [17]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540047 entries, 0 to 2540046
Data columns (total 46 columns):
 #   Column            Dtype  
---  ------            -----  
 0   sport             object 
 1   dsport            object 
 2   proto             object 
 3   state             object 
 4   dur               float64
 5   sbytes            float64
 6   dbytes            float64
 7   sttl              float64
 8   dttl              float64
 9   sloss             float64
 10  dloss             float64
 11  service           object 
 12  Sload             float64
 13  Dload             float64
 14  Spkts             float64
 15  Dpkts             float64
 16  swin              float64
 17  dwin              float64
 18  stcpb             float64
 19  dtcpb             float64
 20  smeansz           float64
 21  dmeansz           float64
 22  trans_depth       float64
 23  res_bdy_len       float64
 24  Sjit              float64
 25  Djit              float64
 26  Stime         

Adding clean up

In [18]:
X_train, X_test, y_train, y_test = train_test_split(combined_data.drop(columns=['attack_cat']),combined_data['attack_cat'],test_size=0.2,random_state=500)

In [23]:
X_train.shape

(2032037, 45)

In [19]:
def create_model():
  model = Sequential()
  model.add(Reshape((-1,1), input_shape=(45,)))
  model.add(Conv1D(32, 3, activation='relu', padding='causal'))
  model.add(Conv1D(64, 3, activation='relu', padding='causal'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(70,recurrent_dropout=0.1))
  model.add(Flatten())
  model.add(Dense(10,activation='softmax'))
  model.summary()
  return model

In [20]:
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 44, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 44, 32)            128       
                                                                 
 conv1d_1 (Conv1D)           (None, 44, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1  (None, 22, 64)            0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 70)                37800     
                                                                 
 flatten (Flatten)           (None, 70)                0         
                                                        

In [21]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])


In [22]:
model.fit(X_train, y_train, epochs=50, batch_size=2048)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
res = model.predict(test_data, batch_size=2048)
def confusion_plot(roi,res,title=None, class_name=None):
    fig, ax = plot_confusion_matrix(conf_mat = confusion_matrix(roi,res), class_names = class_name, show_absolute=True, show_normed=True, hide_spines=False, hide_ticks=False, figsize=(10,10))
    plt.title(title+'\n Accuary=%f \nPrecision=%f \nRecall=%f \nF1_Score=%f '%(\
    accuracy_score(roi,res),\
    precision_score(roi,res, average='macro'),\
    recall_score(roi,res, average='macro'),\
    f1_score(roi,res, average='macro')\
))
    plt.show()
confusion_plot(res.argmax(axis=-1), test_res.argmax(axis=-1), 'Hybrid-Deep CNN-WDLSTM model', data_43_type)